In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset (y, psi, omega) assuming 10000 rows
num_samples = 10000
n, d = 150, 512  # Dimensions given in problem

y = torch.randn(num_samples, n, 1)
psi = torch.randn(num_samples, n, d)
omega = torch.randn(num_samples, n, 1)

# Compute Least Squares Estimate h_LS using Moore-Penrose pseudo-inverse
h_LS = torch.linalg.pinv(psi) @ (y - omega)  # Shape: (num_samples, d, 1)
h_LS = h_LS.squeeze(-1)  # Shape: (num_samples, d)

# Prepare dataset
train_dataset = TensorDataset(h_LS, h_LS)  # Input and output are both h_LS
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define candidate operations for the search space
OPS = {
    'conv_3x3': lambda C: nn.Conv2d(C, C, kernel_size=3, stride=1, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv2d(C, C, kernel_size=5, stride=1, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv2d(C, C, 1, stride=1, bias=False), nn.BatchNorm2d(C)),
    'zero': lambda C: nn.ZeroPad2d(0),
}

# Mixed operation layer with probability-based selection
class MixedOp(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops)))  # Learnable probabilities
    
    def forward(self, x):
        weights = F.softmax(self.alphas, dim=0)  # Normalize with softmax
        return sum(w * op(x) for w, op in zip(weights, self.ops))

# Searchable Neural Network with architecture selection
class SearchNetwork(nn.Module):
    def __init__(self, C, num_layers=3):
        super().__init__()
        self.layers = nn.ModuleList([MixedOp(C) for _ in range(num_layers)])
        self.output_layer = nn.Conv2d(C, C, kernel_size=1, stride=1, bias=False)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.output_layer(x)

# Training function with truncated reverse-mode AD for NAS
def train(model, train_loader, arch_optimizer, model_optimizer, criterion, unroll_steps=1):
    model.train()
    for x, y in train_loader:
        x, y = x.unsqueeze(1), y.unsqueeze(1)  # Add channel dimension for CNN layers
        x, y = x.to(device), y.to(device)
        
        # Compute loss and update model parameters
        model_optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        model_optimizer.step()
        
        # Architecture update using truncated differentiation
        arch_optimizer.zero_grad()
        with torch.no_grad():
            temp_model = SearchNetwork(d).to(device)
            temp_model.load_state_dict(model.state_dict())
        for _ in range(unroll_steps):
            temp_output = temp_model(x)
            temp_loss = criterion(temp_output, y)
            temp_loss.backward()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_layers = 3
num_epochs = 20

# Initialize model and optimizers
model = SearchNetwork(d).to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model with NAS
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

# Extract final architecture based on highest probability operations
def extract_final_architecture(model):
    final_ops = []
    for layer in model.layers:
        best_op_idx = torch.argmax(layer.alphas).item()
        best_op_name = list(OPS.keys())[best_op_idx]
        final_ops.append(best_op_name)
    return final_ops

final_architecture = extract_final_architecture(model)
print("Final Architecture:", final_architecture)

# Save the best model
torch.save(model.state_dict(), "best_model.pth")

print("Neural architecture search completed.")